In [1]:
#load the original ADR related People data
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import pandas as pd

df = pd.read_csv('randomly_generated_data.csv')

In [2]:
df

,person_id,item_list,label
0,0,['K31.8'],1
1,1,"['1732', '1831', '3123002', '3125501', '4019',...",1
2,2,"['E87', 'I08', 'I10', 'I27', 'I48', 'I50', 'I6...",1
3,3,"['13706-02', '47009-00', '49318-00', '92514-39...",1
4,4,"['I48', 'I50', 'I70', 'M80', 'R42', 'Z92', 'Z9...",1
...,...,...,...
4854,91,"['96200-00', '96200-00', '96200-00', 'Z51.1', ...",0
4855,1501,['Z45.81'],0
4856,903,"['M45.02', 'M45.04', 'M45.08', 'M45.02']",0
4857,1276,"['13100-08', '13100-08', '13100-08', '13100-08...",0


In [3]:
from gensim.models import Word2Vec

# Initialize the model
model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)

# Build the vocabulary
model.build_vocab(df['item_list'])

# Train the model
model.train(df['item_list'], total_examples=model.corpus_count, epochs=model.epochs)


(1736418, 9989880)

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Convert item lists to vectors
def convert_to_vector(item_list):
    if len(item_list) > 0:
        return np.mean([model.wv[word] for word in item_list], axis=0)
    else:
        return np.zeros(model.vector_size)  # return an empty vector if item_list is empty

df['vector'] = df['item_list'].apply(convert_to_vector)


In [5]:
df

,person_id,item_list,label,vector
0,0,['K31.8'],1,"[-0.1042064, 0.22137798, -0.37011963, 0.200235..."
1,1,"['1732', '1831', '3123002', '3125501', '4019',...",1,"[0.03814807, 0.40642664, -0.06637045, 0.106025..."
2,2,"['E87', 'I08', 'I10', 'I27', 'I48', 'I50', 'I6...",1,"[-0.14029576, 0.47451043, -0.16005331, -0.0298..."
3,3,"['13706-02', '47009-00', '49318-00', '92514-39...",1,"[0.013342241, 0.29942247, 0.056784254, 0.00096..."
4,4,"['I48', 'I50', 'I70', 'M80', 'R42', 'Z92', 'Z9...",1,"[-0.092795834, 0.41382232, -0.13631779, 0.1648..."
...,...,...,...,...
4854,91,"['96200-00', '96200-00', '96200-00', 'Z51.1', ...",0,"[0.06274616, 0.3131512, 0.13927072, -0.0438267..."
4855,1501,['Z45.81'],0,"[-0.0002693534, 0.29829222, -0.42055616, 0.124..."
4856,903,"['M45.02', 'M45.04', 'M45.08', 'M45.02']",0,"[0.11171655, 0.21888793, -0.25649709, 0.138578..."
4857,1276,"['13100-08', '13100-08', '13100-08', '13100-08...",0,"[0.044825476, 0.34191805, 0.14123091, 0.078840..."


In [9]:
df['label'] = df['label'].astype(int)

X = np.array(df['vector'].tolist())
y = df['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a random forest classifier on the training set
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predict the labels on the test set
y_pred = clf.predict(X_test)

# Print the accuracy
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.8045267489711934


In [10]:
pip install fasttext


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-macosx_10_16_x86_64.whl size=304536 sha256=5e60478bfb67fc8f6e686ce77bf3e92d4e08765e60af850031e3355938040c45
  Stored in directory: /Users/tom/Library/Caches/pip/wheels/3f/84/86/c63cf501c46fb575152daee4b937075a5e9b31765c0e620fd4
Successfully built fasttext
Note: you may need to restart the kernel to use updated packages.


In [11]:
import numpy as np
import fasttext
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Concatenate all words in 'item_list' to form a sentence
df['sentence'] = df['item_list'].apply(lambda x: ' '.join(x))

# Save these sentences to a text file
with open('sentences.txt', 'w') as f:
    for sentence in df['sentence']:
        f.write(sentence + '\n')

# Train a FastText model
model = fasttext.train_unsupervised('sentences.txt', dim=100)

# Use the FastText model to convert item lists to vectors
def convert_to_vector(item_list):
    return np.mean([model.get_word_vector(word) for word in item_list], axis=0)

df['vector'] = df['item_list'].apply(convert_to_vector)

df['label'] = df['label'].astype(int)

X = np.array(df['vector'].tolist())
y = df['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a random forest classifier on the training set
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predict the labels on the test set
y_pred = clf.predict(X_test)

# Print the accuracy
print('Accuracy:', accuracy_score(y_test, y_pred))


Read 1M words
Number of words:  59
Number of labels: 0
Progress: 100.0% words/sec/thread: 1469657 lr:  0.000000 avg.loss:  2.614872 ETA:   0h 0m 0s


Accuracy: 0.8189300411522634


In [12]:
pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 2.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.8/400.8 kB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 3.5 MB/s eta 0:00:0000:0100:010m
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming 'df' is loaded as before
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Define a function to get BERT embeddings
def get_bert_embedding(sentence):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Get the output from BERT
    with torch.no_grad():
        outputs = model(**inputs)
    
    # The first element of the output tuple is the last hidden state, i.e., the embeddings
    # We will take the mean of the embeddings across tokens as the representation for the sentence
    embeddings = outputs[0].mean(dim=1).numpy()
    
    return embeddings[0]

# Concatenate all words in 'item_list' to form a sentence
df['sentence'] = df['item_list'].apply(lambda x: ' '.join(x))

# Use the BERT model to convert item lists to vectors
df['vector'] = df['sentence'].apply(get_bert_embedding)

df['label'] = df['label'].astype(int)

X = np.array(df['vector'].tolist())
y = df['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a random forest classifier on the training set
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predict the labels on the test set
y_pred = clf.predict(X_test)

# Print the accuracy
print('Accuracy:', accuracy_score(y_test, y_pred))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Accuracy: 0.8415637860082305


In [16]:
import pickle

# Assuming the BERT embeddings are stored in the 'vector' column of the 'df' dataframe
bert_embeddings = df['vector'].tolist()

# Save the embeddings to a file
with open('bert_embeddings.pkl', 'wb') as file:
    pickle.dump(bert_embeddings, file)


In [17]:
with open('bert_embeddings.pkl', 'rb') as f:
    X_bert = pickle.load(f)


In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming 'df' is loaded as before
df['label'] = df['label'].astype(int)

# Concatenate all words in 'item_list' to form a sentence
df['sentence'] = df['item_list'].apply(lambda x: ' '.join(x))

# Initialize a TfidfVectorizer
vectorizer = TfidfVectorizer()

# Use the TfidfVectorizer to convert item lists to vectors
X = vectorizer.fit_transform(df['sentence']).toarray()
y = df['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a random forest classifier on the training set
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predict the labels on the test set
y_pred = clf.predict(X_test)

# Print the accuracy
print('Accuracy:', accuracy_score(y_test, y_pred))


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Convert the embeddings list to a NumPy array
X_bert = np.array(df['vector'].tolist())
y = df['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_bert, y, test_size=0.2, random_state=42)

# KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('KNN Accuracy:', accuracy_score(y_test, y_pred))

# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Logistic Regression Accuracy:', accuracy_score(y_test, y_pred))

# SVM
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print('SVM Accuracy:', accuracy_score(y_test, y_pred))


KNN Accuracy: 0.7901234567901234
Logistic Regression Accuracy: 0.8261316872427984
SVM Accuracy: 0.8148148148148148


In [20]:
from gensim.models import Word2Vec

# Initialize the model
model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)

# Build the vocabulary
model.build_vocab(df['item_list'])

# Train the model
model.train(df['item_list'], total_examples=model.corpus_count, epochs=model.epochs)

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Convert item lists to vectors
def convert_to_vector(item_list):
    if len(item_list) > 0:
        return np.mean([model.wv[word] for word in item_list], axis=0)
    else:
        return np.zeros(model.vector_size)  # return an empty vector if item_list is empty

df['vector'] = df['item_list'].apply(convert_to_vector)

df['label'] = df['label'].astype(int)

X = np.array(df['vector'].tolist())
y = df['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('KNN Accuracy:', accuracy_score(y_test, y_pred))

# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Logistic Regression Accuracy:', accuracy_score(y_test, y_pred))

# SVM
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print('SVM Accuracy:', accuracy_score(y_test, y_pred))


KNN Accuracy: 0.7366255144032922
Logistic Regression Accuracy: 0.7417695473251029
SVM Accuracy: 0.772633744855967


In [22]:
import numpy as np
import fasttext
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Concatenate all words in 'item_list' to form a sentence
df['sentence'] = df['item_list'].apply(lambda x: ' '.join(x))

# Save these sentences to a text file
with open('sentences.txt', 'w') as f:
    for sentence in df['sentence']:
        f.write(sentence + '\n')

# Train a FastText model
model = fasttext.train_unsupervised('sentences.txt', dim=100)

# Use the FastText model to convert item lists to vectors
def convert_to_vector(item_list):
    return np.mean([model.get_word_vector(word) for word in item_list], axis=0)

df['vector'] = df['item_list'].apply(convert_to_vector)

df['label'] = df['label'].astype(int)

X = np.array(df['vector'].tolist())
y = df['label']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('KNN Accuracy:', accuracy_score(y_test, y_pred))

# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Logistic Regression Accuracy:', accuracy_score(y_test, y_pred))

# SVM
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print('SVM Accuracy:', accuracy_score(y_test, y_pred))


Read 1M words
Number of words:  59
Number of labels: 0
Progress: 100.0% words/sec/thread: 1386119 lr:  0.000000 avg.loss:  2.615921 ETA:   0h 0m 0s


KNN Accuracy: 0.7592592592592593
Logistic Regression Accuracy: 0.7006172839506173
SVM Accuracy: 0.7438271604938271
